<a href="https://colab.research.google.com/github/pablogadhi/TranslationDetector/blob/master/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/pablogadhi/TranslationDetector
!python -m spacy download en
!python -m spacy download es
%cd TranslationDetector
!pip install --upgrade torch torchtext

Cloning into 'TranslationDetector'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 85 (delta 44), reused 58 (delta 25), pack-reused 0
Unpacking objects: 100% (85/85), done.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 16.2MB 1.2MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=e97efbf9ce2cac58603c16dfb4c833b4f403f24cd734bddbc7aff38b79b3ba58
  Stored in directory: /tmp/pip-ephem-wheel-cache-rikwnu56/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!git pull

Already up to date.


In [0]:
import torch
from translation.train import train_model
from translation.data_loader import load_data, make_iters
from translation.transformer import Transformer

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
!nvidia-smi

Thu Jun 11 17:51:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    12W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load data and create the model

In [0]:
DATA_DIR='/content/drive/My Drive/NLP/data/'

In [10]:
SRC, TGT, train, valid, test = load_data(
        DATA_DIR + "en-es-0_", "en.txt", "es.txt", DATA_DIR + "SRC_Field.pt", DATA_DIR + "TGT_Field.pt")

Loading data...
Data loaded!


In [0]:
train_itr, valid_itr, test_itr = make_iters(
        train, valid, test, device, batch_size=4000)
model = Transformer(len(SRC.vocab), len(TGT.vocab)).to(device)

In [0]:
len(SRC.vocab.itos)

40000

### Train the model

In [12]:
train_model(model, train_itr, valid_itr, SRC, TGT, device, 10, save_at=1, checkpoint_f=DATA_DIR + "checkpoint.pt")

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Step    50 : 637.76 ms/batch | loss  7.90
Step   100 : 603.73 ms/batch | loss  6.86
Step   150 : 626.49 ms/batch | loss  5.98
Step   200 : 630.16 ms/batch | loss  5.56
Step   250 : 626.25 ms/batch | loss  5.30
Step   300 : 633.51 ms/batch | loss  4.92
Step   350 : 632.51 ms/batch | loss  4.89
Step   400 : 630.21 ms/batch | loss  4.66
Step   450 : 635.45 ms/batch | loss  4.73
Step   500 : 631.66 ms/batch | loss  4.30
Step   550 : 633.43 ms/batch | loss  4.36
Step   600 : 626.58 ms/batch | loss  4.36
Step   650 : 626.53 ms/batch | loss  3.98
Step   700 : 625.12 ms/batch | loss  4.11
Step   750 : 631.54 ms/batch | loss  4.37
Step   800 : 630.59 ms/batch | loss  4.28
Step   850 : 634.83 ms/batch | loss  4.19
Step   900 : 634.47 ms/batch | loss  4.25
Step   950 : 631.35 ms/batch | loss  4.06
Step  1000 : 625.05 ms/batch | loss  3.75
Step  1050 : 634.59 ms/batch | loss  4.22
Step  1100 : 626.64 ms/batch | loss  3.68
Step  1150 : 631.85 ms/batch | loss  4.02
Step  1200 : 626.60 ms/batch | los